# MIMIC-IV query API tutorial

This notebook shows examples of how to use the cyclops.query API on [MIMIC-IV v2.0](https://physionet.org/content/mimiciv/2.0/).

Each query is limit to 100 rows (for quick results).

* First, setup the MIMIC-IV database according to the instructions in [mimic-code](https://github.com/MIT-LCP/mimic-code/tree/main/mimic-iv/buildmimic/postgres).
* The database is assumed to be hosted using postgres. Update the config parameters such as username and password, passed to `MIMICIVQuerier` accordingly.

## Imports and instantiate `MIMICIVQuerier`

In [1]:
"""MIMICIV query API tutorial."""

import cyclops.query.ops as qo
from cyclops.query import MIMICIVQuerier


querier = MIMICIVQuerier(
    dbms="postgresql",
    port=5432,
    host="localhost",
    database="mimiciv-2.0",
    user="postgres",
    password="pwd",
)
# List all schemas.
querier.list_schemas()

/home/amritk/.cache/pypoetry/virtualenvs/pycyclops-mhx6UJW0-py3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2023-09-19 11:59:15,401 INFO cyclops.query.orm - Database setup, ready to run queries!


['fhir_etl',
 'fhir_trm',
 'information_schema',
 'mimic_fhir',
 'mimiciv_derived',
 'mimiciv_ed',
 'mimiciv_hosp',
 'mimiciv_icu',
 'mimiciv_note',
 'public']

## Example 1. Get all patient admissions from 2021 or later (approx year of admission)

In [2]:
patients = querier.patients()
admissions = querier.mimiciv_hosp.admissions()
patient_admissions = patients.join(admissions, on="subject_id")
ops = qo.Sequential(
    qo.AddDeltaColumn(["admittime", "dischtime"], years="anchor_year_difference"),
    qo.ConditionAfterDate("admittime", "2021-01-01"),
)
patient_admissions = patient_admissions.ops(ops).run(limit=100)
print(f"{len(patient_admissions)} rows extracted!")

2023-09-19 11:59:17,601 INFO cyclops.query.orm - Query returned successfully!


2023-09-19 11:59:17,602 INFO cyclops.utils.profile - Finished executing function run_query in 0.182561 s


100 rows extracted!


## Example 2. Get all patient encounters with diagnoses (`schizophrenia` in ICD-10 long title), in the year 2015.

In [3]:
patients = querier.patients()
admissions = querier.mimiciv_hosp.admissions()
patient_admissions = patients.join(admissions, on="subject_id")
ops = qo.Sequential(
    qo.AddDeltaColumn(["admittime", "dischtime"], years="anchor_year_difference"),
    qo.ConditionInYears("admittime", "2015"),
)
patient_admissions = patient_admissions.ops(ops)
diagnoses = querier.diagnoses()
diagnoses_ops = qo.Sequential(
    qo.ConditionEquals("icd_version", 10),
    qo.ConditionSubstring("long_title", "schizophrenia"),
)
diagnoses = diagnoses.ops(diagnoses_ops)
patient_admissions_diagnoses = patient_admissions.join(
    join_table=diagnoses,
    on=["subject_id", "hadm_id"],
).run(limit=100)
print(f"{len(patient_admissions_diagnoses)} rows extracted!")

2023-09-19 11:59:18,400 INFO cyclops.query.orm - Query returned successfully!


2023-09-19 11:59:18,400 INFO cyclops.utils.profile - Finished executing function run_query in 0.760419 s


100 rows extracted!


## Example 3. Advanced - uses `ConditionRegexMatch` from `cyclops.query.ops`. Get all patient encounters with diagnoses (ICD-9 long title contains `schizophrenia` and `chronic` ), in the year 2015.

In [4]:
patients = querier.patients()
admissions = querier.mimiciv_hosp.admissions()
patient_admissions = patients.join(admissions, on="subject_id")
ops = qo.Sequential(
    qo.AddDeltaColumn(["admittime", "dischtime"], years="anchor_year_difference"),
    qo.ConditionInYears("admittime", "2015"),
)
patient_admissions = patient_admissions.ops(ops)
diagnoses = querier.diagnoses()
diagnoses_ops = qo.Sequential(
    qo.ConditionEquals("icd_version", 9),
    qo.ConditionRegexMatch("long_title", r"(?=.*schizophrenia)(?=.*chronic)"),
)
diagnoses = diagnoses.ops(diagnoses_ops)
patient_admissions_diagnoses = patient_admissions.join(
    join_table=diagnoses,
    on=["subject_id", "hadm_id"],
).run(limit=100)
print(f"{len(patient_admissions_diagnoses)} rows extracted!")

2023-09-19 11:59:20,036 INFO cyclops.query.orm - Query returned successfully!


2023-09-19 11:59:20,037 INFO cyclops.utils.profile - Finished executing function run_query in 1.526170 s


82 rows extracted!


## Example 4. Get routine vital signs for patients from year 2015.

In [5]:
patients = querier.patients()
admissions = querier.mimiciv_hosp.admissions()
patient_admissions = patients.join(admissions, on="subject_id")
ops = qo.Sequential(
    qo.AddDeltaColumn(["admittime", "dischtime"], years="anchor_year_difference"),
    qo.ConditionInYears("admittime", "2015"),
)
patient_admissions = patient_admissions.ops(ops)
chart_events = querier.chartevents()
vitals = chart_events.ops(qo.ConditionEquals("category", "Routine Vital Signs"))
patient_admissions_vitals = patient_admissions.join(
    join_table=vitals,
    on=["subject_id", "hadm_id"],
).run(limit=100)
print(f"{len(patient_admissions_vitals)} rows extracted!")

2023-09-19 12:00:38,995 INFO cyclops.query.orm - Query returned successfully!


2023-09-19 12:00:38,997 INFO cyclops.utils.profile - Finished executing function run_query in 78.914878 s


100 rows extracted!


## Example 5. Get hemoglobin lab tests for patients from year 2009.

In [6]:
patients = querier.patients()
admissions = querier.mimiciv_hosp.admissions()
patient_admissions = patients.join(admissions, on="subject_id")
ops = qo.Sequential(
    qo.AddDeltaColumn(["admittime", "dischtime"], years="anchor_year_difference"),
    qo.ConditionInYears("admittime", "2009"),
)
patient_admissions = patient_admissions.ops(ops)
chart_events = querier.chartevents()
labs = chart_events.ops(qo.ConditionEquals("label", "hemoglobin"))
patient_admissions_labs = patient_admissions.join(
    join_table=labs,
    on=["subject_id", "hadm_id"],
).run(limit=100)
print(f"{len(patient_admissions_labs)} rows extracted!")

2023-09-19 12:01:47,563 INFO cyclops.query.orm - Query returned successfully!


2023-09-19 12:01:47,564 INFO cyclops.utils.profile - Finished executing function run_query in 68.524319 s


100 rows extracted!


## Example 6. Get radiology reports and filter on keywords `lymphadenopathy` and `infectious` occurring together from year 2009.

In [7]:
patients = querier.patients()
admissions = querier.mimiciv_hosp.admissions()
patient_admissions = patients.join(admissions, on="subject_id")
ops = qo.Sequential(
    qo.AddDeltaColumn(["admittime", "dischtime"], years="anchor_year_difference"),
    qo.ConditionInYears("admittime", "2009"),
)
patient_admissions = patient_admissions.ops(ops)
radiology_notes = querier.mimiciv_note.radiology()
radiology_notes_ops = qo.Sequential(
    qo.And(
        qo.ConditionLike("text", "% lymphadenopathy %"),
        qo.ConditionLike("text", "% infectious %"),
    ),
)
radiology_notes = radiology_notes.ops(radiology_notes_ops)
patient_admissions_radiology_notes = patient_admissions.join(
    join_table=radiology_notes,
    on=["subject_id", "hadm_id"],
).run(limit=100)
print(f"{len(patient_admissions_radiology_notes)} rows extracted!")

2023-09-19 12:01:54,873 INFO cyclops.query.orm - Query returned successfully!


2023-09-19 12:01:54,874 INFO cyclops.utils.profile - Finished executing function run_query in 7.285099 s


100 rows extracted!


## Example 7. Get all female patient encounters from year 2015, and return as dask dataframe (lazy evaluation) with 4 partitions (batches) aggregated based on `subject_id`.

In [8]:
patients = querier.patients()
admissions = querier.mimiciv_hosp.admissions()
patient_admissions = patients.join(admissions, on="subject_id")
ops = qo.Sequential(
    qo.AddDeltaColumn(["admittime", "dischtime"], years="anchor_year_difference"),
    qo.ConditionInYears("admittime", "2015"),
    qo.Cast("gender", "str"),
    qo.ConditionEquals("gender", "F"),
)
patient_admissions = patient_admissions.ops(ops)
patient_admissions = patient_admissions.run(
    backend="dask",
    index_col="subject_id",
    n_partitions=4,
)
print(f"{len(patient_admissions)} rows extracted!")
print(f"Return type: {type(patient_admissions)}")
print(f"Number of partitions: {patient_admissions.npartitions}")

2023-09-19 12:01:55,290 INFO cyclops.query.orm - Query returned successfully!


2023-09-19 12:01:55,291 INFO cyclops.utils.profile - Finished executing function run_query in 0.383649 s


35639 rows extracted!
Return type: <class 'dask.dataframe.core.DataFrame'>
Number of partitions: 4


## Example 8. Running a raw SQL string.

In [9]:
data = querier.db.run_query("SELECT * FROM mimiciv_hosp.admissions LIMIT 100")
print(f"{len(data)} rows extracted!")

2023-09-19 12:01:56,271 INFO cyclops.query.orm - Query returned successfully!


2023-09-19 12:01:56,272 INFO cyclops.utils.profile - Finished executing function run_query in 0.007934 s


100 rows extracted!
